# 20 min segments, 25 clusters similarity matrix generation

In [18]:
import pandas as pd
import numpy as np
import pickle

In [73]:
flattened = pickle.load(open('./model2_20min_partition_clusters_flattened', 'rb'))
print('The shape of the data set is: ', flattened.shape)
flattened.head()

The shape of the data set is:  (3201180, 1)


cluster_label
psn timestamp                         
34  2015-12-05 00:00:00             18
    2015-12-05 00:10:00             18
    2015-12-05 00:10:00             18
    2015-12-05 00:20:00             18
    2015-12-05 00:20:00             18

Transform the dataframe above into a matrix which is m x n (PSN number x cluster_label) where each cell is a count of how many time segments for a PSN fell into the cluster_label.

In [74]:
flattened = flattened.reset_index()
flattened = pd.DataFrame(flattened.groupby(['psn', 'cluster_label'])['timestamp'].count()).reset_index()
flattened = flattened.pivot(index='psn', columns='cluster_label', values='timestamp').fillna(0)

In [78]:
print('This is the matrix that will be fed into the pairwise distance function.')
flattened.head()

This is the matrix that will be fed into the pairwise distance function.


cluster_label,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
psn,,,,,,,,,,,,,,,,,,,,,
34,0.0,22810.0,0.0,28262.0,0.0,49440.0,0.0,0.0,60.0,0.0,...,0.0,78078.0,0.0,26488.0,24.0,0.0,0.0,238.0,0.0,708.0
35,4688.0,34.0,0.0,5506.0,20114.0,88.0,0.0,33506.0,0.0,4.0,...,0.0,436.0,0.0,26.0,26.0,30430.0,14800.0,4.0,0.0,22910.0
36,10024.0,36.0,0.0,1926.0,22532.0,46.0,0.0,29736.0,0.0,0.0,...,0.0,136.0,0.0,40.0,14.0,28980.0,14212.0,2.0,0.0,16472.0
37,4852.0,3134.0,530.0,8588.0,13460.0,7260.0,10.0,16902.0,8.0,888.0,...,42.0,8990.0,1224.0,3352.0,152.0,16844.0,15634.0,54.0,424.0,14340.0
38,24038.0,0.0,2.0,0.0,2762.0,2.0,0.0,10.0,0.0,0.0,...,0.0,0.0,6.0,0.0,0.0,28.0,7626.0,2.0,0.0,0.0


In [83]:
from sklearn.metrics import pairwise
similarityMatrix = pd.DataFrame(pairwise.pairwise_distances(flattened, metric = 'euclidean'), columns=flattened.index, index=flattened.index)
similarityMatrix.head()

psn,34,35,36,37,38,39,40,41,42,45,...,62,63,64,65,66,67,68,69,71,72
psn,,,,,,,,,,,,,,,,,,,,,
34,0.000000,116007.894611,115580.528083,96310.101152,108805.757201,105585.251584,102961.558263,91162.472235,92451.586293,110189.673672,...,106951.232064,101805.399346,68235.474088,68553.403679,62874.373412,67490.648952,95295.675201,103178.908290,103910.985675,116911.384595
35,116007.894611,0.000000,11221.174448,27812.143319,60105.230621,56299.213174,59638.548641,59327.563678,42989.057817,26143.073040,...,65018.927675,57403.150401,76254.470767,69086.156761,81346.144322,79644.236628,60350.354796,48964.837343,46262.321991,19374.331988
36,115580.528083,11221.174448,0.000000,25475.525981,53148.228795,50252.839522,56568.590790,56464.635233,49179.355831,23072.524959,...,62183.931558,54317.154638,74350.124707,67619.600206,79542.094365,77549.380552,66479.803730,45453.150606,47178.216414,21507.938069
37,96310.101152,27812.143319,25475.525981,0.000000,40842.844661,37987.737179,42778.519750,39109.272251,46195.383233,25225.729246,...,50579.025218,40225.747277,55357.578596,47018.198307,60536.849340,59245.160883,56278.144177,32342.864004,34684.746273,36537.640646
38,108805.757201,60105.230621,53148.228795,40842.844661,0.000000,19143.654406,39014.713225,39063.385312,80483.097803,51759.312360,...,46746.654041,35792.608511,62477.451453,54996.007091,68493.858995,65986.520320,82133.855090,35260.987337,45100.033792,64812.851195


In [85]:
pickle.dump(similarityMatrix, open( "model2_20min_partition_clusters_similarityMatrix", "wb" ) )

Use the one-liner below to get a sorted_dictionary for similarity.

In [87]:
PSN = 34
similarityMatrix[PSN].drop([PSN], axis=0).sort_values(ascending=True).to_dict()

{66: 62874.3734123848,
 67: 67490.64895228078,
 64: 68235.47408789654,
 65: 68553.40367917555,
 41: 91162.47223501565,
 42: 92451.58629250231,
 68: 95295.67520092399,
 37: 96310.10115247518,
 63: 101805.39934600718,
 47: 102808.3028748165,
 50: 102828.58534473767,
 51: 102879.3943800215,
 40: 102961.55826326639,
 46: 102966.67398726639,
 69: 103178.90829040595,
 61: 103437.74829335758,
 71: 103910.98567524033,
 39: 105585.2515837321,
 60: 105845.62934764949,
 53: 106206.48294713463,
 62: 106951.23206396456,
 38: 108805.75720061875,
 45: 110189.67367226386,
 58: 115154.95527331857,
 36: 115580.52808323727,
 35: 116007.89461066863,
 72: 116911.38459534213,
 57: 122099.61267751835,
 59: 124993.87525795015,
 49: 126312.20384428419,
 48: 127867.94062625706,
 56: 132901.19341826843,
 55: 133461.77893314624}